In [58]:
import numpy as np
import random

class Person:
    def __init__(self):
        self.recover_prob = 0.2
        self.die_prob = 0.05
        self.init_sick_prob = 0.1
        self.recovered = False
        self.dead = False
        self.sick = False
        self.vaccinated = False
        self.infect_others_prob = 0.05 #infectin rate
        self.average_meetups = 10
        self.days_sick=0
        
        self.init_sick_or_not()
        
    def init_sick_or_not(self):
        """When created each person starts as either sick or healthy"""
        prob = random.random() #The returned value is 0-1: https://www.w3schools.com/python/ref_random_random.asp
        if prob <= self.init_sick_prob:
            # This should yield that about 10% of the people created are sick, the rest healthy
            self.sick = True
        else:
            self.sick = False
    
    def day_passes(self, population, init_scenario=False):
        """This method describes what happends to each person each day"""
        
        # If a person is sick, they can infect others.
        #prob = random.random()
        if self.sick:
            self.days_sick+=1
            if init_scenario==False: #On the 1st the sick isn't infecting yet.
                self.infect_others(population)
        
        # Will person recover?
        prob = random.random()
        if prob <= self.recover_prob and self.sick == True:
            # Person has rehabilitated and is now healthy!
            self.sick = False
            self.recovered = True 
        
        # If person is still sick they might die
        prob = random.random()
        if prob <= self.die_prob and self.sick == True:
            self.dead = True
            self.sick = False
            
    def infect_others(self, population):
        person_encounters = random.sample(range(population.size), k= self.average_meetups)
        for person_id in person_encounters:
            prob = random.random()
            if prob <= self.infect_others_prob and population[person_id].recovered==False and population[person_id].dead==False and population[person_id].vaccinated==False:
            # Person has rehabilitated and is now healthy!
                population[person_id].sick = True
             

In [59]:
class Village:
    def __init__(self, init_population_size):
        self.population = np.empty(init_population_size, Person)#https://www.w3resource.com/numpy/array-creation/empty.php. Question: Class Person as dtype? Why isn't it according to the syntax empty(size, dtype=Person)
        self.init_vaccination = 0.2*init_population_size
        self.vaccination_started = False
        self.daily_vaccination_threshold = 0.04*init_population_size
        self.generate_inhabitants()
    
    def generate_inhabitants(self):
        """Generates the population"""
        for i in range(self.population.size):
            self.population[i] = Person()

    def vaccinate_population(self):
        people_vaccinated_today=0
        for person in self.population:
            if people_vaccinated_today == self.daily_vaccination_threshold: break
            elif person.sick == False and person.dead== False and person.vaccinated == False:
                person.vaccinated == True
                people_vaccinated_today+=1

    def advance_days(self, init_scenario = False):
        """Counts the status of the citizens in the community"""
        people_sick = 0
        people_recovered = 0
        people_dead = 0
        people_vaccinated = 0
        people_immune = 0
        for person in self.population:
            if person.sick: people_sick += 1     
            if person.dead: people_dead += 1
            if person.recovered and person.vaccinated:
                people_recovered += 1
                people_vaccinated += 1
                people_immune += 1
            elif person.vaccinated:
                people_vaccinated += 1
                people_immune += 1
            elif person.recovered:
                people_recovered += 1 
                people_immune += 1
            person.day_passes(self.population, init_scenario)
        
        people_susceptible = self.population.size - (people_immune + people_dead + people_sick)

        if people_sick>=self.init_vaccination and self.vaccination_started == False:
            self.vaccination_started = True
            print(f"At the end of the day {people_sick} became sick and vaccination process has been initiated.")
        if self.vaccination_started: self.vaccinate_population()
        return people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible

    def start_simulation(self):
        """This function controls the simulation and what happends in a day"""
        current_day = 0
        
        people_sick, people_recovered, people_vaccinated, people_immune, people_dead, people_susceptible= self.advance_days(init_scenario = True)
        while people_sick:
            print(f"By day {current_day} {people_sick} people are sick, {people_dead} are dead and {people_recovered} has recovered")
            current_day += 1
            people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible = self.advance_days()
            Patient_with_duration=[person.days_sick for person in self.population] #List conprehension
        
    


        people_unaffected = self.population.size - (people_sick + people_recovered + people_dead)
        print(f"\nBy day {current_day} {people_sick} people are sick, {people_dead} are dead and {people_recovered} has recovered. {people_unaffected} people were never in contact with the virus")
        print("The village has recovered and the virus is eliminated!")
        print("The longest time an individual was sick is: ", max(Patient_with_duration), "days")
        print(f"In total {people_vaccinated} people recieved vaccination and {people_susceptible} remain susceptible to the virus.")

In [62]:
def main():
    pop_size = 1000
    village = Village(pop_size)
    village.start_simulation()

main()

By day 0 106 people are sick, 0 are dead and 0 has recovered
By day 1 118 people are sick, 2 are dead and 18 has recovered
By day 2 128 people are sick, 9 are dead and 35 has recovered
By day 3 148 people are sick, 18 are dead and 61 has recovered
By day 4 164 people are sick, 23 are dead and 92 has recovered
By day 5 187 people are sick, 29 are dead and 126 has recovered
At the end of the day 200 became sick and vaccination process has been initiated.
By day 6 200 people are sick, 35 are dead and 167 has recovered
By day 7 214 people are sick, 46 are dead and 196 has recovered
By day 8 230 people are sick, 52 are dead and 238 has recovered
By day 9 234 people are sick, 62 are dead and 279 has recovered
By day 10 220 people are sick, 65 are dead and 335 has recovered
By day 11 197 people are sick, 75 are dead and 375 has recovered
By day 12 184 people are sick, 88 are dead and 411 has recovered
By day 13 163 people are sick, 97 are dead and 453 has recovered
By day 14 150 people are si